# Background Subtraction Algorithm for Cloud Masking

## Initialize the Earth Engine API

The following cell grants the notebook access to Google's Earth Engine using your account. Note that your account must have permission to use the API.

In [ ]:
import ee
import geemap.core as geemap
import numpy as np


# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
try:
  ee.Initialize(project='examplegeostat')
except:
  ee.Initialize(project='geostat-test')

imageCoordinates = [[-96.8, 28.0],
[-85.3, 28.0],
[-85.3, 22],
[-96.8, 22]]
polygonArea = ee.Geometry.Polygon(imageCoordinates);

## Defining Area of Interest

Current Plan: Examine the Gulf of Mexico

In [ ]:
# Band aliases.
BLUE = 'CMI_C01'
RED = 'CMI_C02'
VEGGIE = 'CMI_C03'
GREEN = 'GREEN'
CIRRUS = 'CMI_C04'
CO2 = 'CMI_C16'
CLOUDS = 'CMI_C11'

# 16 pairs of CMI and DQF followed by Bah 2018 synthetic green.
# Band numbers in the EE asset, 0-based.
NUM_BANDS = 33

# Skipping the interleaved DQF bands.
BLUE_BAND_INDEX = (1 - 1) * 2
RED_BAND_INDEX = (2 - 1) * 2
VEGGIE_BAND_INDEX = (3 - 1) * 2
GREEN_BAND_INDEX = NUM_BANDS - 1

# Visualization range for GOES RGB.
GOES_MIN = 0.0
GOES_MAX = 0.7  # Alternatively 1.0 or 1.3.
GAMMA = 1.3

goesClouds = {
    'bands': [CIRRUS, CO2, CLOUDS],
    'min': [0,92,127],
    'max': [1.3,319,342],
    'gamma': [1,0.5,2]
}

goesRgbViz = {
    'bands': [RED,VEGGIE,BLUE],
    'min': 0,
    'max': 1.3,
    'gamma': 1
}

def apply_scale_and_offset(image):
    image = ee.Image(image)
    bands = [None] * NUM_BANDS
    for i in range(1, 17):
        band_name = 'CMI_C' + str(100 + i)[-2:]
        offset = ee.Number(image.get(band_name + '_offset'))
        scale = ee.Number(image.get(band_name + '_scale'))
        bands[(i - 1) * 2] = image.select(band_name).multiply(scale).add(offset)

        dqf_name = 'DQF_C' + str(100 + i)[-2:]
        bands[(i - 1) * 2 + 1] = image.select(dqf_name)

    # Bah, Gunshor, Schmit, Generation of GOES-16 True Color Imagery without a
    # Green Band, 2018. https://doi.org/10.1029/2018EA000379
    # Green = 0.45 * Red + 0.10 * NIR + 0.45 * Blue
    green1 = bands[RED_BAND_INDEX].multiply(0.45)
    green2 = bands[VEGGIE_BAND_INDEX].multiply(0.10)
    green3 = bands[BLUE_BAND_INDEX].multiply(0.45)
    green = green1.add(green2).add(green3)
    bands[GREEN_BAND_INDEX] = green.rename(GREEN)
    return ee.Image(ee.Image(bands).copyProperties(image, image.propertyNames()))


startDate = ee.Date('2020-08-27T10:00:00');
endDate = startDate.advance(15, 'hour');
goesCollection = ee.ImageCollection('NOAA/GOES/16/MCMIPF').filterDate(startDate, endDate)

ids = goesCollection.aggregate_array('system:index').getInfo()

for image_id in ids:
    collection = 'NOAA/GOES/16/MCMIPF/'
    asset_id = collection + image_id
    image = apply_scale_and_offset(asset_id)
    #   print(image.getInfo())

    map = geemap.Map(center=[25,-89],zoom=6,add_google_map=False,
                 search_ctrl=False,zoom_ctrl=False,draw_ctrl=False,scale_ctrl=False)

    rgbmap = geemap.Map(center=[25,-89],zoom=6,add_google_map=False,
                 search_ctrl=False,zoom_ctrl=False,draw_ctrl=False,scale_ctrl=False)

    rgbmap.add_layer(
            image, goesRgbViz
    )
    #display(rgbmap)

    map.addLayer(image, goesClouds, None, False);

    polygonImage = image.clip(polygonArea);

    cloudsMask = polygonImage.select('CMI_C13').lt(280);
    cloudsImage = polygonImage.updateMask(cloudsMask);
    map.addLayer(cloudsImage, goesClouds, 'Clouds Only');
    map.centerObject(polygonArea,6);

    display(map);

Map(center=[25.085830488741014, -91.05000000000001], controls=(ZoomControl(options=['position', 'zoom_in_text'…

Map(center=[25.085830488741014, -91.05000000000001], controls=(ZoomControl(options=['position', 'zoom_in_text'…

Map(center=[25.085830488741014, -91.05000000000001], controls=(ZoomControl(options=['position', 'zoom_in_text'…

Map(center=[25.085830488741014, -91.05000000000001], controls=(ZoomControl(options=['position', 'zoom_in_text'…

Map(center=[25.085830488741014, -91.05000000000001], controls=(ZoomControl(options=['position', 'zoom_in_text'…

Map(center=[25.085830488741014, -91.05000000000001], controls=(ZoomControl(options=['position', 'zoom_in_text'…

Map(center=[25.085830488741014, -91.05000000000001], controls=(ZoomControl(options=['position', 'zoom_in_text'…

KeyboardInterrupt: 

In [ ]:
image.propertyNames()

In [ ]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd_HH:mm:ss').getInfo()

'2020-08-27_11:10:20'

In [ ]:
goesCat = ee.ImageCollection('NOAA/GOES/16/MCMIPF')
goesCat.propertyNames()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
goesCat = ee.ImageCollection('NOAA/GOES/16/MCMIPF').sort('date_range').filterDate('2020-08-27T10:00:00', '2020-08-27T12:00:00')
ids = goesCat.aggregate_array('system:index').getInfo()

original_map = geemap.Map(center=[25,-89],zoom=6,add_google_map=False,
                 search_ctrl=False,zoom_ctrl=False,draw_ctrl=False,scale_ctrl=False)

layer_map = geemap.Map(center=[25,-89],zoom=6,add_google_map=False,
                 search_ctrl=False,zoom_ctrl=False,draw_ctrl=False,scale_ctrl=False)

projection = {'crs':'EPSG:3857','transform':[1,0,-179,0,-1,89]}

for image_id in ids:
    collection = 'NOAA/GOES/16/MCMIPF/'
    asset_id = collection + image_id
    image = apply_scale_and_offset(asset_id)
    genName = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd_HH:mm:ss').getInfo()
    polygonImage = image.clip(polygonArea);

    cloudsMask_final = polygonImage.select('CMI_C13').lt(290);
    cloudsImage_final = polygonImage.updateMask(cloudsMask);

    whole_image_name = 'Whole_' + genName
    mask_image_name = 'Mask_' + genName
    info = cloudsImage_final.getInfo()
    crsGOES= info['bands'][0]['crs']
    crs_transform = info['bands'][0]['crs_transform']

    #task = ee.batch.Export.image.toDrive(
     #   image=polygonImage,
      #  description= whole_image_name,
       # crs=projection['crs'],
        #crsTransform=projection['transform'],
       # region=polygonArea,
       # folder = 'drive/MyDrive/ENEE439D Group Project/Cloud Data'
    #)

    task = ee.batch.Export.image.toDrive(
        image=cloudsImage_final,
        description= mask_image_name,
        crs=crsGOES,
        crsTransform=crs_transform,
        region=polygonArea,
        folder = 'ENEE439D Group Project/Cloud Data'
    )

    task.start()

In [ ]:
image.select('CMI_C04').projection().getInfo()

{'type': 'Projection',
 'wkt': 'PROJCS["unnamed", \n  GEOGCS["unknown", \n    DATUM["unknown", \n      SPHEROID["Spheroid", 6378137.0, 298.2572221]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["GEOS"], \n  PARAMETER["central_meridian", -75.0], \n  PARAMETER["satellite_height", 35786023.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["sweep", 0.0], \n  PARAMETER["Option", 0.0], \n  UNIT["m", 1.0], \n  AXIS["x", EAST], \n  AXIS["y", NORTH]]',
 'transform': [2004.017315487541,
  0,
  -5434894.700982174,
  0,
  2004.017315487541,
  -5434895.218222249]}

In [ ]:
image1 = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')
image1.select('B3').projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:32610',
 'transform': [30, 0, 460785, 0, -30, 4264215]}

# Thresholding the Whole Image for Possible Detection Model
https://developers.google.com/earth-engine/apidocs

The Idea is to make a model that will learn to mask the clouds. Using the whole image and the clipped rgb image to train the model.

In [ ]:
#clouds_Mask_Cirrus = image.select(CIRRUS).gt(0.0025)
clouds_Mask_Clouds = image.select(CLOUDS).lt(275)
clouds_Mask_CO2 = image.select(CO2).lt(250)
#clouds_Mask_New = clouds_Mask_Cirrus.bitwiseAnd(clouds_Mask_Clouds).bitwiseAnd(clouds_Mask_CO2)
clouds_Mask_New = clouds_Mask_Clouds.bitwiseAnd(clouds_Mask_CO2)

clouds_Image_Cirrus = image.updateMask(clouds_Mask_New)

newmap = geemap.Map(center=[25,-89],zoom=6,add_google_map=False,
                 search_ctrl=False,zoom_ctrl=False,draw_ctrl=False,scale_ctrl=False)

newmap.addLayer(image, goesRgbViz,'RGB',False)
newmap.addLayer(clouds_Image_Cirrus, goesClouds, 'Clouds Only',False)
newmap.addLayer(clouds_Image_Cirrus, goesRgbViz,'Clouds Only RGB')

display(newmap)

Map(center=[25, -89], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…